# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=123456")
cur = conn.cursor()

In [3]:
# get all json files from current folder
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
# relative path (./: no need)
song_files = get_files('data/song_data')

In [5]:
# check format 
filepath = song_files[0]

# print to check (absolute path)
print(filepath)

C:\Users\Big City Boy\Desktop\Modeling_BT1\Project1\data\song_data\A\A\A\TRAAAAW128F429D538.json


In [6]:
# read data from 1 filepath in song_files
first_song = pd.read_json(filepath, lines= True)
first_song.iloc[0]

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                    NaN
artist_longitude                   NaN
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
Name: 0, dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# collect data from all json files to concatenate into 'songs' dataframe
songs = pd.concat([pd.read_json(path, lines=True) for path in song_files], ignore_index=True) # reorganize the index

# Select columns for song ID, title, artist ID, year, and duration (info related to songs)
modified_songs = songs[['song_id', 'title', 'artist_id', 'year', 'duration']]

# convert all values from dataframe modified_songs (in array type) to a list 
song_data = modified_songs.values.tolist()
song_data[:3]

[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179],
 ['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546],
 ['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
for song in song_data:
    try:
        cur.execute(song_table_insert, song)
        conn.commit()
    except psycopg2.Error as e:
        print("Error data", song)
        print(e)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
# Select columns for song ID, title, artist ID, year, and duration (info related to songs)
modified_artists = songs[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

# convert all values from dataframe modified_songs (in array type) to a list 
artist_data = modified_artists.values.tolist()
artist_data[:5]

[['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan],
 ['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892],
 ['ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan],
 ['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan],
 ['ARXR32B1187FB57099', 'Gob', '', nan, nan]]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
for artist in artist_data:
    try:
        cur.execute(artist_table_insert, artist)
        conn.commit()
    except psycopg2.Error as e:
        print("Error data")
        print(e)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath1 = log_files[0]

print(filepath1)

C:\Users\Big City Boy\Desktop\Modeling_BT1\Project1\data\log_data\2018\11\2018-11-01-events.json


In [13]:
first_artist = pd.read_json(filepath1, lines= True)
first_artist.iloc[0]

artist                                                        None
auth                                                     Logged In
firstName                                                   Walter
gender                                                           M
itemInSession                                                    0
lastName                                                      Frye
length                                                         NaN
level                                                         free
location                         San Francisco-Oakland-Hayward, CA
method                                                         GET
page                                                          Home
registration                                         1540919166796
sessionId                                                       38
song                                                          None
status                                                        

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
# concatenate data from all log json files into a datframe 'actions'
actions = pd.concat([pd.read_json(path, lines=True) for path in log_files], ignore_index=True)

# filter by NextSong action
actions_NextSong = actions[actions.page == 'NextSong']

# convert ts timestamp column to datetime
time = pd.to_datetime(actions_NextSong.ts, unit='ms')
time.values

array(['2018-11-01T21:01:46.796000000', '2018-11-01T21:05:52.796000000',
       '2018-11-01T21:08:16.796000000', ...,
       '2018-11-14T23:26:53.796000000', '2018-11-14T23:32:07.796000000',
       '2018-11-14T23:38:06.796000000'], dtype='datetime64[ns]')

In [15]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column 
# and set time_data to a list containing these values in order

time_data = (time.values, time.dt.hour.values, time.dt.day.values, time.dt.week.values, 
             time.dt.month.values, time.dt.year.values, time.dt.weekday.values)

# Specify labels for these columns and set to column_labels
column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']

# Create a dataframe, time_df, containing the time data for all files by combining column_labels 
# and time_data into a dictionary and converting this into a dataframe

time_df= pd.DataFrame(data=list(zip(*time_data)), columns=column_labels)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3
3,2018-11-01 21:11:13.796,21,1,44,11,2018,3
4,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [16]:
for i, row in time_df.iterrows():
    try:
        cur.execute(time_table_insert, list(row))
        conn.commit()
    except psycopg2.Error as e:
        print("Error data", row)
        print(e)
    

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [17]:
user_df = actions[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head(3)

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [18]:
for i, row in user_df.iterrows():
    try: 
        cur.execute(user_table_insert, row)
        conn.commit()
    except psycopg2.Error as e:
        print("Error data", row)
        print(e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [19]:
actions['ts'] = pd.to_datetime(actions.ts, unit='ms')
actions.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [20]:
for index, row in actions.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, 
                    row.sessionId, row.location, row.userAgent)
    try:
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    except psycopg2.Error as e:
        print("Error data", row)
        print(e)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [21]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.